In [1]:
from src.classes import *
import pandas as pd
import argparse
import os
import re
import pickle
from datetime import date

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('-f','--folder', type=str, help='folder containing ST dataframes',
                    default='../../data/ST_files/ST_matrix_STARsolo_PetterREFs_ensembl/')
parser.add_argument('-i', '--image_path', type=str, help='folder containing image data',
                    default='../../../Images_rev1/')
parser.add_argument('-s', '--sample_list', type=str, help='folder containing ST samples to process', 
                    default='../../data/ST_files/TX-test.txt')
parser.add_argument('-o', '--output', type=str, help='folder to be used as an output',
                    default='../../results/st_pp/')
parser.add_argument('--save_separate', default=1, help='save files for each samples separately')
# parser.add_argument('--mode', default='client')
# parser.add_argument('--host', default='127.0.0.1')
# parser.add_argument('--port', default=37237)
args = parser.parse_args()
args.folder = '../../data/ST_files/ST_matrix_STARsolo_PetterREFs_ensembl/'
args

Namespace(folder='../../data/ST_files/ST_matrix_STARsolo_PetterREFs_ensembl/', image_path='../../../Images_rev1/', sample_list='../../data/ST_files/TX-test.txt', output='../../results/st_pp/', save_separate=1)

In [3]:
os.getcwd()


'/media/Dropbox/MNM projects/Spatial transcriptomics project/Data analysis/Complete_ST_pipeline/bin/st_analysis'

In [4]:
if args.sample_list != 'None':
    samples = pd.read_csv(args.sample_list, header=None)
    samples = samples[0].to_list()
    print('The following samples will be used')
    print(' '.join(samples))
else:
    print('All samples in the ST folder will be used')
    samples = [re.search('[STCN]+[0-9]+_[C-E][1-2]', x).group(0) for x in os.listdir(args.folder) if 'stdata' in x]
    print(' '.join(samples))

sample_list = []
for sample in samples:
    print(f'Loading sample {sample}')
    adata = STLoader()
    adata.load_stdata(args.folder, sample)
    adata.add_image(args.image_path)
    adata.correct_feature_position(args.image_path)
    adata.filter_by_expression()
    adata = adata.anndata
    sample_list.append(adata)

adata_st = ST_Anndata()
adata_st.concat_anndata(sample_list, samples)

The following samples will be used
ST3_C2 CN56_E2
Loading sample ST3_C2
dataset loaded
       ENSG00000238009  ENSG00000177757  ENSG00000187634  ENSG00000188976  \
5x14               0.0              0.0              0.0              0.0   
5x11               0.0              0.0              0.0              0.0   
16x14              0.0              0.0              0.0              0.0   
8x15               0.0              0.0              0.0              0.0   
11x24              0.0              0.0              0.0              0.0   
...                ...              ...              ...              ...   
6x13               0.0              0.0              0.0              0.0   
10x31              0.0              0.0              0.0              0.0   
14x3               0.0              0.0              0.0              0.0   
19x21              0.0              0.0              0.0              0.0   
6x24               0.0              0.0              0.0          

/home/lcadmin/miniconda3/envs/ST_pipeline/lib/python3.9/site-packages/anndata/_core/anndata.py:798: UserWarning: 
AnnData expects .obs.index to contain strings, but got values like:
    [0, 1, 2, 3, 4]

    Inferred to be: integer

  value_idx = self._prep_dim_index(value.index, attr)


Filtering features and genes by expression. Positions with less than 300 transcripts are discarded as well as genes that do not reach at least 2 in at least 4 features
3 features and 27316 genes dropped
Loading sample CN56_E2
dataset loaded
       ENSG00000237491  ENSG00000272438  ENSG00000187634  ENSG00000188976  \
5x14               0.0              0.0              0.0              0.0   
15x3               0.0              1.0              0.0              0.0   
26x9               0.0              0.0              0.0              0.0   
16x16              0.0              0.0              0.0              0.0   
2x28               0.0              0.0              0.0              0.0   
...                ...              ...              ...              ...   
23x30              0.0              0.0              1.0              0.0   
3x33               0.0              0.0              0.0              0.0   
20x13              0.0              0.0              0.0          

/home/lcadmin/miniconda3/envs/ST_pipeline/lib/python3.9/site-packages/anndata/_core/anndata.py:798: UserWarning: 
AnnData expects .obs.index to contain strings, but got values like:
    [0, 1, 2, 3, 4]

    Inferred to be: integer

  value_idx = self._prep_dim_index(value.index, attr)


Filtering features and genes by expression. Positions with less than 300 transcripts are discarded as well as genes that do not reach at least 2 in at least 4 features
15 features and 27781 genes dropped
<class 'anndata._core.anndata.AnnData'>


In [6]:
adata_st.anndata

AnnData object with n_obs × n_vars = 1484 × 11931
    obs: 'feature', 'orig_feature', 'array_row', 'array_col', 'in_tissue', 'sample', 'slide', 'x', 'y', 'new_x', 'new_y', 'pixel_x', 'pixel_y', 'n_transcripts'
    uns: 'sample', 'merged', 'spatial'
    obsm: 'spatial'

In [22]:
adata_backup = adata_st.anndata.copy()
adata_backup.write_h5ad('adatatest.h5ad')

In [4]:
adata_backup = sc.read_h5ad('adatatest.h5ad')

In [5]:
adata_sub = ST_Anndata()
adata_sub.anndata = adata_backup[:, :4000].copy()

In [ ]:
adata_st.anndata = merge_gene_symbol_duplicates(adata_st.anndata, symbol_column='gene_id')
print(adata_st.anndata)
print(type(adata_st))
adata_st.QC()
adata_st = adata_st.anndata

In [6]:
if 'gene_id' not in adata_sub.anndata.var.columns:
    varx = adata_sub.transalate_ensembl()

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
                                                                 name   symbol
query                                                                         
ENSG00000001084           glutamate-cysteine ligase catalytic subunit     GCLC
ENSG00000002746     HECT, C2 and WW domain containing E3 ubiquitin...    HECW1
ENSG00000004487                                 lysine demethylase 1A    KDM1A
ENSG00000004534                           RNA binding motif protein 6     RBM6
ENSG00000004866                       suppression of tumorigenicity 7      ST7
...                                                               ...      ...
ENSRNOG00000005157  RB binding protein 7, chromatin remodeling factor    Rbbp7
ENSRNOG00000005158                  solute carrier family 24 member 5  Slc24a5
ENSRNOG00000005175              sphingosine-1-phosphate phosphatase 1    Sgpp1
ENSRNOG00000005176        

In [7]:
varx

,ensembl_id,name,gene_id,species
gene_id,,,,
GCLC,ENSG00000001084,glutamate-cysteine ligase catalytic subunit,GCLC,ENSG
HECW1,ENSG00000002746,"HECT, C2 and WW domain containing E3 ubiquitin...",HECW1,ENSG
KDM1A,ENSG00000004487,lysine demethylase 1A,KDM1A,ENSG
RBM6,ENSG00000004534,RNA binding motif protein 6,RBM6,ENSG
ST7,ENSG00000004866,suppression of tumorigenicity 7,ST7,ENSG
...,...,...,...,...
RBBP7,ENSRNOG00000005157,"RB binding protein 7, chromatin remodeling factor",Rbbp7,ENSRNOG
SLC24A5,ENSRNOG00000005158,solute carrier family 24 member 5,Slc24a5,ENSRNOG
SGPP1,ENSRNOG00000005175,sphingosine-1-phosphate phosphatase 1,Sgpp1,ENSRNOG


In [ ]:
len([gene for gene in varx.index if isNaN(gene)])

In [ ]:
varx[varx['gene_id'].isna()]

In [8]:
adata_sub.anndata

View of AnnData object with n_obs × n_vars = 1484 × 3851
    obs: 'feature', 'orig_feature', 'array_row', 'array_col', 'in_tissue', 'sample', 'slide', 'x', 'y', 'new_x', 'new_y', 'pixel_x', 'pixel_y', 'n_transcripts'
    var: 'ensembl_id', 'name', 'gene_id', 'species'
    uns: 'merged', 'sample', 'spatial'
    obsm: 'spatial'

In [15]:
var = adata_backup.var
var

,ensembl_id,name,gene_id,species
gene_id,,,,
GCLC,ENSG00000001084,glutamate-cysteine ligase catalytic subunit,GCLC,ENSG
HECW1,ENSG00000002746,"HECT, C2 and WW domain containing E3 ubiquitin...",HECW1,ENSG
KDM1A,ENSG00000004487,lysine demethylase 1A,KDM1A,ENSG
RBM6,ENSG00000004534,RNA binding motif protein 6,RBM6,ENSG
ST7,ENSG00000004866,suppression of tumorigenicity 7,ST7,ENSG
...,...,...,...,...
AC105662.3,ENSRNOG00000060851,NaN,AC105662.3,ENSRNOG
LOC100910996,ENSRNOG00000060879,uncharacterized LOC100910996,LOC100910996,ENSRNOG
LOC120101127,ENSRNOG00000061021,U4 spliceosomal RNA,LOC120101127,ENSRNOG


In [ ]:
def merge_gene_symbol_duplicates(adata, symbol_column='gene_id'):
    adata.var["value"] = 0
    var = adata.var
    original_len = len(var)
    X = adata.X
    temp_df = pd.DataFrame(X).transpose()

    # establish gene names without variants
    gene_no_alt = [x.split('.')[0].upper() for x in var[symbol_column]]

    # add the no alternaticve splicing variants into teh var dataframe and make it the index
    var['Gene_no_alt'] = gene_no_alt
    var.index = var['Gene_no_alt']

    temp_df[symbol_column] = gene_no_alt # add the no splicing variant column into the expression dataframe
    temp_df = temp_df.groupby(symbol_column).sum().transpose()


    var = var.drop_duplicates(subset=['Gene_no_alt'])
    var = var.reindex(temp_df.columns) # make sure that the grouping by some miracle did not rearrange the gene positions

    new_len = len(var)

    adata.uns['merged'] = True
    ad_merge = ad.AnnData(X = temp_df.iloc[:, :].to_numpy(),
                                   var = var,
                                   obs = adata.obs,
                                   obsm = adata.obsm,
                                   uns = adata.uns)
    print(f'Scaled from {original_len} genes incl. alternative splicing to {new_len} genes without alternative splicing variants')

    return ad_merge
